In [1]:
import os
import shutil
import pandas as pd
import numpy as np
# !pip3 install addressparser
import addressparser as ap

# rename all the excels to year-index and copy to a new folder

In [ ]:
basedir = "/Users/sun/Desktop/Econ580Data/"
movedir = basedir + "all/"

# clean all the files in all first 
if os.path.isdir(movedir):
    shutil.rmtree(movedir)
os.mkdir(movedir)

# used to grant permission
# permission_array = []
# Walk through all files in the directory that contains the files to copy to the "all" folder
for root, dirs, files in os.walk(basedir):
    # use data from 1999 to 2012
    if not root[-4:].isdigit():
        continue
    if int(root[-4:]) < 2000 or int(root[-4:]) > 2012:
        continue
    
    index = 1
    for file in files:
        ori_file = os.path.abspath(root) + "/" + file
        base, extension = os.path.splitext(file)

        if extension != ".xls":
            continue
        # file year-index.xls
        new_file = movedir + root[-4:] + "-" + str(index) + extension
        
        index += 1
        # break
        shutil.copy(ori_file, new_file)
        # permission_array.append(new_file)

print("Done")
# print("\"" + "\",  _\n\"""".join(permission_array) + "\"" )

# xls2xlsx, in vba file
references: 
1. translate xls to xlsx: https://answers.microsoft.com/en-us/msoffice/forum/msoffice_excel-mso_other/batch-convert-xls-to-xlsx/1d9b3d78-daf0-4014-8fb2-930aca6493b0
2. request permission for all: https://docs.microsoft.com/en-us/office/vba/office-mac/grantaccesstomultiplefiles


# Rename, restructure, format all the files

## Rename

In [ ]:
# remove the xls file if it is already converted to xlsx
basedir = "/Users/sun/Desktop/Econ580Data/"
movedir = basedir + "all/"

# Walk through all files in the directory that contains the files to copy to the "all" folder
for file in os.listdir(movedir):
    # should only have xls and xlsx
    ori_file = movedir + file
    base, extension = os.path.splitext(file)

    if extension != ".xlsx":
        continue
    # is now filename.xlsx
    # remove the xls based on xlsx file
    # print(movedir + file[:-1])
    if os.path.exists(movedir + file[:-1]):
        # print("exist")
        os.remove(movedir + file[:-1]) 
    else:
        # print("xls not exist: ", movedir + file[:-1])
        pass
    
# check whether there are xls file not converted to xlsx
xls_file = [f for f in os.listdir(movedir) if os.path.splitext(f)[1] == ".xls"]
print(xls_file)

## Restructure

In [ ]:
# Format
basedir = "/Users/sun/Desktop/Econ580Data/"
movedir = basedir + "all/"
outdir = basedir + "formated/"

# 2000
file_path = movedir+"2000-2.xlsx"
xls = pd.ExcelFile(file_path)
# out_path = outdir + "2000.csv"

for year in range(2012,2013):
    file_province = {} # used to check xlsx file if format error
    all_data = []
    out_path = outdir + str(year) + ".csv"
    for cur_file in os.listdir(movedir):
        cur_provinces = [] # add to province dic

        # should only have xls and xlsx
        ori_file = movedir + cur_file
        base, extension = os.path.splitext(cur_file)

        if extension != ".xlsx":
            continue
        
        # if "2012-16.xlsx" not in cur_file or "~$" in cur_file:
        if str(year) not in cur_file or "~$" in cur_file:
            continue

        # open file and parse
        print(cur_file)

        file_path = movedir+ cur_file
        xls = pd.ExcelFile(file_path)           
        # Iterate through worksheets in opened Excel file
        for sheet in xls.sheet_names:
            # Create a Pandas dataframe from the Excel worksheet (with no headers)
            dfs = pd.read_excel(file_path, sheet_name=sheet, header=None)
            if year == 2012:
                # worksheet for year 2012 are vertically appended, others are appended horizontally
                list_index = dfs[dfs.isnull().all(1)].index.tolist()
                df_list = np.split(dfs, list_index, axis = 0)
                # remove all null column
                df_list = [df_list[x] for x in range(len(df_list)) if not df_list[x].isnull().all().all()]               
            else:
                list_index = [x for x in range(dfs.shape[1]) if dfs[:][x].isnull().all(0) ]
                # separated by 2 null column
                df_list = np.split(dfs, list_index, axis = 1)
                # remove all null column
                df_list = [df_list[x] for x in range(len(df_list)) if not df_list[x].isnull().all().all()]
                # print(len(df_list))
            
            for df in df_list:
                # remove null column and row
                df = df.dropna(how="all", axis=0).dropna(how="all", axis=1).values.tolist()
                # filter out wrong df
                if len(df) < 30:
                    print("wrong sheet:", df)
                    continue
                # strip all the elements
                df = [[str(x).strip() if x!= np.nan else "" for x in y] for y in df]
                # first line would be year e.g. 2000年县(市)社会经济主要指标
                
                if "经济主要" in df[0][0]:
                    # check province whether in the first line
                    if df[0][0].strip()[0].isdigit():
                        df = df[1:]
                        province = df[0][0]
                    else:
                        # province in this line
                        # e.g 新疆维吾尔自治区 2006年县(市)社会经济主要指标统计
                        privince = df[0][0].strip().split(" ")[0]

                # province in [0][0], get province, remove first line
                # get county, start in second line, might change line in between
                #  e.g.['    指标' '  单位' '    桐庐县' '    淳安县']
                # print(df)
                county_end = [x for x in range(len(df)) if "一、" in df[x][0]]
                if county_end != []:
                    cur = df[1]
                    for i in range(2, county_end[0]):
                        cur = [ (cur[x] + df[i][x]).replace("nan", "").replace("\n","") for x in range(len(cur)) ]
                    counties = cur[2:]
                    df = df[county_end[0]:]
                    df = np.delete(df, 1, 1).tolist()
                else:
                    # this case: split between the worksheet
                    # e.g. 2009-17, sheet 9, 13, 16, no index name in the left
                    county_end = [x for x in range(len(df)) if df[x][0].strip().isdigit()][0]
                    counties = df[0]
                    for i in range(2, county_end):
                        counties = [ (counties[x] + df[i][x]).replace("nan", "").replace("\n","") for x in range(len(cur)) ]
                    df = df[county_end:]

                counties = [x.replace("nan", "").replace("\n","").replace("\r","") for x in counties]    
                # print(province, counties)
                # unit in second column, remove column
                df = [y for y in df if ( ("、" not in "".join(y) and "，" not in "".join(y)) or "医院、卫生院床位数" in "".join(y)) and y[0] != "" ]
                # df = pd.DataFrame(data=df).replace("nan", np.nan).dropna(how="any", axis=0).values.tolist()
                # add province and to the front
                df.insert(0, ["县"] + counties)
                df.insert(0, ["省"] + [province for x in range(len(counties))] )
                df.insert(0, ["年"] + [year for x in range(len(counties))])
                # transpose
                df = np.transpose(df).tolist()
                # print(df)

                # add to all_data
                if all_data != []:
                    # already have
                    all_data += df[1:]
                else:
                    all_data = df
                
                # add to provinces
                cur_provinces += [province]

        # add to province dic
        file_province[cur_file] = np.unique(cur_provinces).tolist()
    
    # print(file_province)

    # # print(np.array(all_data).shape)
    # print("\n".join([",".join(x) for x in all_data]))
    # # save to csv
    with open(out_path, "a+") as f:
        f.write("\n".join([",".join(x) for x in all_data])) 

In [120]:
# for year 2000 to 2002, one more column than others: 国内生产总值
# remove and save to another
# Format
basedir = "/Users/sun/Desktop/Econ580Data/"
outdir = basedir + "formated/"

for year in [2000,2001]:
    file_path = outdir + str(year) + ".csv"
    df = pd.read_csv(file_path)
    print(df.shape)
    # print(df)
    df = df.drop(["国内生产总值"], axis=1)
    print(df.shape)
    # make a duplicate of the origin file
    os.rename(file_path, outdir + str(year) + "_ori.csv")
    df.to_csv(outdir + str(year) + ".csv", index=False)  

# year 2002 one more column "农村用电量"
year = 2002
file_path = outdir + str(year) + ".csv"
df = pd.read_csv(file_path)
print(df.shape)
# print(df)
df = df.drop(["农村用电量"], axis=1)
print(df.shape)
os.rename(file_path, outdir + str(year) + "_ori.csv")
df.to_csv(outdir + str(year) + ".csv", index=False)  

(2081, 34)
(2081, 33)
(2166, 34)
(2166, 33)
(2176, 34)
(2176, 33)


In [150]:
basedir = "/Users/sun/Desktop/Econ580Data/"
outdir = basedir + "formated/"
file_path = basedir+"2000-2012.csv"
file_path_dup = basedir+"2000-2012dup.csv"

# remove the old ones
if os.path.exists(file_path):
    os.remove(file_path) 
if os.path.exists(file_path_dup):
    os.remove(file_path_dup) 

# append all the file together
for year in range(2000,2013):
    cur_file = outdir + str(year) + ".csv"
    
    # read cur_file
    if os.path.exists(cur_file):
        with open(cur_file, "r") as f:
            if year == 2000:
                with open(file_path, "a+") as dst:
                    dst.write(f.read().strip())
                    dst.write("\n")
            else:
                with open(file_path, "a+") as dst:
                    # ignore the vaiable names
                    dst.write( "\n".join(f.read().split("\n")[1:]).strip() )
                    dst.write("\n")
        
        # remove the redundent "\n"
        content = ""
        with open(file_path, "r") as dst:
            content = dst.read()
        with open(file_path, "w") as dst:
            dst.write(content.replace("\n\n","\n"))

# make a copy of the appended file
shutil.copy(file_path, file_path_dup)


'/Users/sun/Desktop/Econ580Data/2000-2012dup.csv'

## Format and Refine

Problems:
1. duplicate lines
2. province not unique (for some lines)
    e.g. 广东省、广西壮族自治区 & 广西壮族自治区
         安徽省、福建省
3. name for county might be different in the end,
    e.g. 广西壮族自治区,邕宁区 & 广西壮族自治区,邕宁县

In [174]:
# load the data, remove duplicate first
basedir = "/Users/sun/Desktop/Econ580Data/"
# movedir = basedir + "all/"
# outdir = basedir + "formated/"

# read the data, use the duplicate one in case of error
file_path = basedir+"2000-2012dup.csv"
csv = pd.read_csv(file_path)

# remove duplicate lines, current no duplicate, 25382 lines
df = pd.DataFrame(csv).drop_duplicates()
print(df.shape)


/Users/sun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(25382, 33)


In [175]:
# First tryout for different county names and remove invalid lines
# print(df.shape)
error_index = []
for i in range(df.shape[0]):
    cur_line = df.iloc[i]
    
    # check whether the county exist in current year
    if i % 1000 == 0:
        print(i)

    try:
        ori_loc = df.iloc[i]["县"]
        # change between 区 and 县
        if "区" in ori_loc:
            changed_loc = ori_loc.replace("区", "县")
        else:
            changed_loc = ori_loc.replace("县","区")
        
        # print(cur_line)
        # find the location of county by its name
        if ap.area_map.get_relational_addrs(ori_loc):
            found_loc = ap.area_map.get_relational_addrs(ori_loc)
        else:
            found_loc = ap.area_map.get_relational_addrs(changed_loc)
        
        # replace to as much as possible
        if len(found_loc) == 1:
            # only one county found, replace
            # at use name of the row instead of row index
            cur_id = df.iloc[i].name 
            df.at[cur_id,"省"] = found_loc[0][0]
            df.at[cur_id,"县"] = found_loc[0][2]
    except:
        # print(i)
        error_index.append(i)

print(df.shape)
print("error lines:", error_index)

# save here
basedir = "/Users/sun/Desktop/Econ580Data/"
file_path = basedir + "2000-2012_pro1.csv"
# remove the old ones
if os.path.exists(file_path):
    os.remove(file_path) 
df.to_csv(file_path, index=False)  

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
(25382, 33)
error lines: [13331, 13332, 13333, 13334, 13335, 17426, 17427, 17489, 17490, 17491, 17492, 17493, 17494, 17495, 17496, 17497, 17498, 17499, 17500, 17501, 17502, 17503, 17504, 17505, 17506, 17507, 17508, 17509, 17510, 17511, 17512, 17513, 17514, 17515, 17516, 17517, 17518, 17519, 19023, 19024, 19025, 19026, 19027, 19277, 19278, 19279, 19280, 19281, 19991]


In [176]:
# remove error indices
basedir = "/Users/sun/Desktop/Econ580Data/"
# read the data, use the pro1 one in case of error
file_path = basedir+"2000-2012_pro1.csv"
df = pd.read_csv(file_path)

# remove duplicate line, size from 25280 here
df = pd.DataFrame(csv).drop_duplicates()
print(df.shape)

# error_index from previous cell, have error in county or province names
# error_index = [13331, 13332, 13333, 13334, 13335, 17426, 17427, 17489, 17490, 17491, 17492, 17493, 17494, 17495, 17496, 17497, 17498, 17499, 17500, 17501, 17502, 17503, 17504, 17505, 17506, 17507, 17508, 17509, 17510, 17511, 17512, 17513, 17514, 17515, 17516, 17517, 17518, 17519, 19024, 19025, 19026, 19027, 19028, 19278, 19279, 19280, 19281, 19282, 19992]
df = df.drop([df.iloc[i].name for i in error_index])

# check again for 省 just in case
index_0 = [i for i in range(df.shape[0]) if (not isinstance(df.iloc[i].at['省'],str)) or (not isinstance(df.iloc[i].at['县'],str))]
# print(df.iloc[17462].name)
# print(df.iloc[17469])
print(index)
df = df.drop([df.iloc[i].name for i in index])

# now 25333 rows
print(df.shape)

# save here
basedir = "/Users/sun/Desktop/Econ580Data/"
file_path = basedir + "2000-2012_pro2.csv"
# remove the old ones
if os.path.exists(file_path):
    os.remove(file_path) 
df.to_csv(file_path, index=False)  

/Users/sun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(25382, 33)
[]
(25333, 33)


In [216]:
# check whether there are still rows with multiple provinces
# remove error indices
basedir = "/Users/sun/Desktop/Econ580Data/"
# read the data, use the pro1 one in case of error
file_path = basedir+"2000-2012_pro2.csv"
df = pd.read_csv(file_path)

print("ori shape:", df.shape)

# check liines with multiple province at a line, check type here just in case
index = [i for i in range(df.shape[0]) if (not isinstance(df.iloc[i].at['省'],str)) or "、" in df.iloc[i].at['省']]

print("to remove: ", len(index))
# still 3375 lines with multiple provinces
df = df.drop([df.iloc[i].name for i in index])

print("cur shape: ", df.shape)
# still have 24937 observations

# save here
basedir = "/Users/sun/Desktop/Econ580Data/"
file_path = basedir + "2000-2012_pro3.csv"
# remove the old ones
if os.path.exists(file_path):
    os.remove(file_path) 
df.to_csv(file_path, index=False) 

/Users/sun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ori shape: (25333, 33)
to remove:  3375
cur shape:  (21958, 33)


In [231]:
# check again, should be null array
basedir = "/Users/sun/Desktop/Econ580Data/"
# read the data, use the pro1 one in case of error
file_path = basedir+"2000-2012_pro3.csv"
df = pd.read_csv(file_path)
# print(df.describe)

# resolve types, to numeric
for col in df.columns:
    if col == '省' or col == '县':
        continue
    df[col] = pd.to_numeric(df[col], errors='coerce')
# df.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

# drop lines with too many nan
df = df.dropna(thresh=10)
# make sure have the the data required here
df = df[df['年'].notna()]
df = df[df['年末总人口'].notna()]
df = df[df['其中：乡村人口'].notna()]

print(df.shape) #21473 here

# save here
basedir = "/Users/sun/Desktop/Econ580Data/"
file_path = basedir + "2000-2012_pro4.csv"
# remove the old ones
if os.path.exists(file_path):
    os.remove(file_path) 
df.to_csv(file_path, index=False) 

/Users/sun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,7,8,9,10,11,13,14,15,16,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(21362, 33)


In [232]:
# check again, should not be type error here any more
basedir = "/Users/sun/Desktop/Econ580Data/"
# read the data, use the pro1 one in case of error
file_path = basedir+"2000-2012_pro4.csv"
df = pd.read_csv(file_path)

In [ ]:
# Add calculated field or other data for analysis

Existiing columns:
年,省,县,行政区域土地面积,乡(镇)个数,村民委员会个数,年末总户数,其中：乡村户数,年末总人口,其中：乡村人口,年末单位从业人员数,乡村从业人员数,其中：农林牧渔业,农业机械总动力,本地电话用户,第一产业增加值,第二产业增加值,地方财政预算内收入,财政支出,城乡居民储蓄存款余额,年末金融机构各项贷款余额,粮食产量,棉花产量,油料产量,肉类总产量,规模以上工业企业数,规模以上工业总产值(现价),基本建设投资完成额,普通中学在校学生数,小学在校学生数,医院、卫生院床位数,社会福利院数,社会福利院床位数

Add now columns for analysis
1. aggregate for each county
2. rural/imtermediate/urban
3. last year's GDP
4. dummy for earthquake area or not

In [241]:
# add rural/urban/immediate
basedir = "/Users/sun/Desktop/Econ580Data/"
# read the data, use the pro1 one in case of error
file_path = basedir+"2000-2012_pro4.csv"
df = pd.read_csv(file_path)

def dummy_type(row):
    # 0rural, 1immediate, 2urban
    return row['其中：乡村人口'] / row['年末总人口']

df['type_index'] = df.apply (lambda row: dummy_type(row), axis=1)
# index = [i for i in range(df.shape[0]) if df.iloc[i].at["type"] > 1]
# print(index)
df.describe()

# save here
basedir = "/Users/sun/Desktop/Econ580Data/"
file_path = basedir + "2000-2012_type_index.csv"
# remove the old ones
if os.path.exists(file_path):
    os.remove(file_path) 
df.to_csv(file_path, index=False)


In [2]:
# aggregate in tableau prep
# show the stats here

# add rural/urban/immediate
basedir = "/Users/sun/Desktop/Econ580Data/"
# read the data, use the pro1 one in case of error
file_path = basedir+"2000-2012after.csv"
df = pd.read_csv(file_path)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617 entries, 0 to 1616
Data columns (total 38 columns):
Resilience        1617 non-null float64
2008就业率           1617 non-null float64
2012就业率           1617 non-null float64
GDP per capita    1617 non-null float64
省                 1617 non-null object
县                 1617 non-null object
年                 1617 non-null float64
行政区域土地面积          1617 non-null float64
乡(镇)个数            1617 non-null float64
村民委员会个数           1617 non-null float64
年末总户数             1617 non-null float64
其中：乡村户数           1617 non-null float64
年末总人口             1617 non-null float64
其中：乡村人口           1617 non-null float64
年末单位从业人员数         1617 non-null float64
乡村从业人员数           1617 non-null float64
其中：农林牧渔业          1617 non-null float64
农业机械总动力           1616 non-null float64
本地电话用户            1616 non-null float64
第一产业增加值           1617 non-null float64
第二产业增加值           1617 non-null float64
地方财政预算内收入         1617 non-null float64
财政支出           

In [39]:
pd.DataFrame(df["医院、卫生院床位数"].describe()).T

,count,mean,std,min,25%,50%,75%,max
医院、卫生院床位数,1617.0,661.245961,472.756043,21.8,329.166667,542.833333,900.166667,3340.166667


In [9]:
# add the earthquake dummy

basedir = "/Users/sun/Desktop/Econ580Data/"
# read the data, use the pro1 one in case of error
file_path = basedir+"2000-2012afterDup.csv"
df = pd.read_csv(file_path)

# print(df)
# df
# Sichuan, Gansu, Shaanxi, Chongqing, Shanxi, Guizhou and Hubei
province_affected = ["四川省","甘肃省","陕西省","重庆市","山西省","贵州省","湖北省"]
earthquake_dummy = []
for i in range(df.shape[0]):
    # print(df.iloc[i]["Province"], df.iloc[i]["Province"] in province_affected)
    if df.iloc[i]["Province"] in province_affected:
        earthquake_dummy.append(1)
    else:
        earthquake_dummy.append(0)
# add one column
df['Earthquake affected dummy'] = earthquake_dummy

# remove year column
df = df.drop(columns=["Year"])
  
# Observe the result
df.info()

# save
file_path = basedir + "2000-2012_all.csv"
# remove the old ones
if os.path.exists(file_path):
    os.remove(file_path) 
df.to_csv(file_path, index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617 entries, 0 to 1616
Data columns (total 38 columns):
Resilience                                                                  1617 non-null float64
2008 Employment Rate                                                        1617 non-null float64
2012 Employment Rate                                                        1617 non-null float64
GDP per capita                                                              1617 non-null float64
Province                                                                    1617 non-null object
County                                                                      1617 non-null object
Area                                                                        1617 non-null float64
Village Number                                                              1617 non-null float64
village committee number                                                    1617 non-null float64
Year end hous

In [ ]:
# current columns:
# Resilience,2008 Employment Rate,2012 Employment Rate,GDP per capita,Province,County,Year,Area,Village Number,village committee number,Year end household number, Among them: Rural household number,Year end population,Among them: rural population,year end industry employment,rural employment,Among them: Agriculture forestry animal husbandry and fishery employment,Total power of agricultural machinery,local phone users,GVA of first sector,GVA of second sector,Local government budget revenue,Local fiscal expenditure,Savings of urban and rural residents,Year end balance of various loans of financial institutions,Grain yield,Cotton yield,Gas yield,Meat yield,Number of industrial enterprises above designated size,Gross industrial output value above designated size,Completed capital construction investment,Number of students enrolled in junior high school,Number of students enrolled in primary school,Number of beds in hospitals and health centers,Number of social welfare institutions,Number of beds in social welfare institutions,rural population share